I will perform the data cleaning and feature engineering steps into the same notebook, due to the fact that there involve less steps than usual: dropping columns with missing values, computing missing values, creating new features etc. In larger projects, they should be split:

Initial analysis <br>
Data cleaning<br>
Feature Engineering <br>
Feature Selection <br>
Pipeline<br>
Prediction run<br>


In [1]:
import pandas as pd
import numpy as np

import regex as re
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick 
import matplotlib.dates as mdates
from matplotlib.ticker import PercentFormatter, FuncFormatter
%matplotlib inline
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
pylab.rcParams.update(params)
from cycler import cycler

import seaborn as sns
sns.set()

import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from textacy import preprocessing
import textacy
from nltk.corpus import stopwords
from nltk.stem import *

import spacy
nlp = spacy.load('en_core_web_sm')

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from  sklearn.metrics  import accuracy_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
# for one hot encoding with sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold

# for the Q-Q plots
import scipy.stats as stats


# for one hot encoding with feature-engine
from feature_engine.encoding import OneHotEncoder as fe_OneHotEncoder

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import roc_auc_score, r2_score, mean_squared_error
# environment settings
pd.set_option('display.max_column',None)
pd.set_option('display.max_rows',None)

In [2]:
#read data
calendar = pd.read_csv('/Users/asyagadzhalova/Documents/GitHub/Boston-Airbnb-data/src/data/raw_data/calendar.csv')
listings =  pd.read_csv('/Users/asyagadzhalova/Documents/GitHub/Boston-Airbnb-data/src/data/raw_data/listings.csv')
reviews = pd.read_csv('/Users/asyagadzhalova/Documents/GitHub/Boston-Airbnb-data/src/data/raw_data/reviews.csv')

From the previous notebook, in which we initially explored the data, we have defined the folling questions to answer:
1. When are the busiest times in Boston? Which are the busiest neghbourhoods? 
I will perform some descriptive statistics and visualisations. 
In order to answer that, we will use the calendar data create occupancy metrics based on calendar data, to understand the occupancy levels throughout the year. Add to this data the neighbourhood information. 
2. What drives/defines the prices of the AirBNB in Boston? Which are the top variables that affect the price? 
We will build a regression model to predict a continuous variable, and to understand the features that drive the model. 
3. What drives the occupancy rates of the AirBNBs in Boston? Which are more engaged than others? What are the factors/variables that define higher occupancy rates? 
We will discretize the occupancy rates into several categories, and then build a multi-class classification model to predict the occupancy rates. 

# 1. Data cleaning

### Check for duplicates, columns with repeated values, drop unnecessary columns

####  Calendar data

In [3]:
calendar.shape

(1308890, 4)

In [4]:
calendar.drop_duplicates(subset=['listing_id','date'],inplace=True)

In [5]:
calendar.shape

(1308525, 4)

In [6]:
calendar['date'] = pd.to_datetime(calendar['date'])
calendar['month'] = calendar['date'].map(lambda x: x.strftime('%Y-%m'))
#calendar['week'] = calendar['date'].map(lambda x: x.strftime("%V"))

In [7]:
cal_month_avail = calendar.groupby(['listing_id','month','available']).agg({'date':'nunique'}).reset_index()

In [8]:
cal_month_total = cal_month_avail.groupby(['listing_id','month']).agg({'date':'sum'}).reset_index()
cal_month_total.rename({'date':'total_days'},inplace=True,axis=1)

In [9]:
cal_month_busy = cal_month_avail[cal_month_avail['available']=='f'].groupby(['listing_id','month']).agg({'date':'sum'}).reset_index()
cal_month_busy.rename({'date':'busy_days'},inplace=True,axis=1)

In [10]:
cal_month_total = cal_month_total.merge(cal_month_busy, how = 'left', on = ['listing_id','month'])

In [11]:
cal_month_total['busy_days'].fillna(0,inplace=True)
cal_month_total['occupancy_metrics'] = cal_month_total['busy_days']/cal_month_total['total_days']*100

In [12]:
cal_metrics_total = cal_month_total.groupby(['listing_id']).agg({'total_days':'sum','busy_days':'sum'}).reset_index()
#cal_metrics_total.sort_values(by='occupancy_metrics',ascending =False).head()

In [13]:
cal_metrics_total['occupancy_metrics'] = round(cal_metrics_total['busy_days']/cal_metrics_total['total_days']*100,0)

In [14]:
cal_metrics_total.head()

,listing_id,total_days,busy_days,occupancy_metrics
0,3353,365,116.0,32.0
1,5506,365,21.0,6.0
2,6695,365,41.0,11.0
3,6976,365,46.0,13.0
4,8792,365,117.0,32.0


In [15]:
cal_metrics_total.shape

(3585, 4)

In [16]:
#20% of the listings are fully booked throughout the year
cal_metrics_total[cal_metrics_total['occupancy_metrics']==100].shape

(705, 4)

In [17]:
cal_metrics_total[cal_metrics_total['occupancy_metrics']<=40].shape

(1660, 4)

In [18]:
calendar.to_pickle('/Users/asyagadzhalova/Documents/GitHub/Boston-Airbnb-data/src/data/processed/calendar.pkl')
cal_month_total.to_pickle('/Users/asyagadzhalova/Documents/GitHub/Boston-Airbnb-data/src/data/processed/calendar_metrics_monthly.pkl')
cal_metrics_total.to_pickle('/Users/asyagadzhalova/Documents/GitHub/Boston-Airbnb-data/src/data/processed/calendar_metrics.pkl')

#### Listings data

In [19]:
listings.describe()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,square_feet,guests_included,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,jurisdiction_names,calculated_host_listings_count,reviews_per_month
count,3.585000e+03,3.585000e+03,3.585000e+03,3585.000000,3585.000000,0.0,3585.000000,3585.000000,3585.000000,3571.000000,3575.000000,3576.000000,56.000000,3585.000000,3585.000000,3.585000e+03,0.0,3585.00000,3585.000000,3585.000000,3585.000000,3585.000000,2772.000000,2762.000000,2767.000000,2765.000000,2767.000000,2763.000000,2764.000000,0.0,0.0,3585.000000,2829.000000
mean,8.440875e+06,2.016091e+13,2.492311e+07,58.902371,58.902371,NaN,42.340032,-71.084818,3.041283,1.221647,1.255944,1.609060,858.464286,1.429847,3.171269,2.872584e+04,NaN,8.64993,21.833194,38.558159,179.346444,19.044630,91.916667,9.431571,9.258041,9.646293,9.646549,9.414043,9.168234,NaN,NaN,12.733891,1.970908
std,4.500787e+06,8.516813e-01,2.292781e+07,171.119663,171.119663,NaN,0.024403,0.031565,1.778929,0.501487,0.753060,1.011745,608.874310,1.056787,8.874133,1.670136e+06,NaN,10.43533,21.860966,33.158272,142.136180,35.571658,9.531686,0.931863,1.168977,0.762753,0.735507,0.903436,1.011116,NaN,NaN,29.415076,2.120561
min,3.353000e+03,2.016091e+13,4.240000e+03,0.000000,0.000000,NaN,42.235942,-71.171789,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000e+00,NaN,0.00000,0.000000,0.000000,0.000000,0.000000,20.000000,2.000000,2.000000,2.000000,4.000000,2.000000,2.000000,NaN,NaN,1.000000,0.010000
25%,4.679319e+06,2.016091e+13,6.103425e+06,1.000000,1.000000,NaN,42.329995,-71.105083,2.000000,1.000000,1.000000,1.000000,415.000000,1.000000,1.000000,3.650000e+02,NaN,0.00000,0.000000,0.000000,19.000000,1.000000,89.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,NaN,NaN,1.000000,0.480000
50%,8.577620e+06,2.016091e+13,1.928100e+07,2.000000,2.000000,NaN,42.345201,-71.078429,2.000000,1.000000,1.000000,1.000000,825.000000,1.000000,2.000000,1.125000e+03,NaN,4.00000,16.000000,37.000000,179.000000,5.000000,94.000000,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000,NaN,NaN,2.000000,1.170000
75%,1.278953e+07,2.016091e+13,3.622147e+07,7.000000,7.000000,NaN,42.354685,-71.062155,4.000000,1.000000,2.000000,2.000000,1200.000000,1.000000,3.000000,1.125000e+03,NaN,15.00000,40.000000,68.000000,325.000000,21.000000,98.250000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,NaN,6.000000,2.720000
max,1.493346e+07,2.016091e+13,9.385411e+07,749.000000,749.000000,NaN,42.389982,-71.000100,16.000000,6.000000,5.000000,16.000000,2400.000000,14.000000,300.000000,1.000000e+08,NaN,30.00000,60.000000,90.000000,365.000000,404.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,NaN,136.000000,19.150000


#### Explore data, remove some unnecessary columns

In [21]:
listings.iloc[:,0:20].head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",NaN,"The bus stop is 2 blocks away, and frequent. B...","You will have access to 2 bedrooms, a living r...",NaN,Clean up and treat the home the way you'd like...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,31303940
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...","If you don't have a US cell phone, you can tex...",Plenty of safe street parking. Bus stops a few...,Apt has one more bedroom (which I use) and lar...,"If I am at home, I am likely working in my hom...",Pet friendly but please confirm with me if the...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,2572247
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,I am in a scenic part of Boston with a couple ...,"PUBLIC TRANSPORTATION: From the house, quick p...","I am living in the apartment during your stay,...","ABOUT ME: I'm a laid-back, friendly, unmarried...","I encourage you to use my kitchen, cooking and...",https://a2.muscache.com/im/pictures/6ae8335d-9...,https://a2.muscache.com/im/pictures/6ae8335d-9...,https://a2.muscache.com/im/pictures/6ae8335d-9...,https://a2.muscache.com/im/pictures/6ae8335d-9...,16701
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,Please be mindful of the property as it is old...,There are buses that stop right in front of th...,The basement has a washer dryer and gym area. ...,We do live in the house therefore might be som...,- The bathroom and house are shared so please ...,https://a2.muscache.com/im/pictures/39764190-1...,https://a2.muscache.com/im/pictures/39764190-1...,https://a2.muscache.com/im/pictures/39764190-1...,https://a2.muscache.com/im/pictures/39764190-1...,6031442
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",I have one roommate who lives on the lower lev...,From Logan Airport and South Station you have...,You will have access to the front and side por...,I love my city and really enjoy sharing it wit...,"Please no smoking in the house, porch or on th...",https://a1.muscache.com/im/pictures/97154760/8...,https://a1.muscache.com/im/pictures/97154760/8...,https://a1.muscache.com/im/pictures/97154760/8...,https://a1.muscache.com/im/pictures/97154760/8...,15396970


In [25]:
listings.iloc[:,20:40].head()

,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed
0,https://www.airbnb.com/users/show/31303940,Virginia,2015-04-15,"Boston, Massachusetts, United States",We are country and city connecting in our deck...,NaN,NaN,NaN,f,https://a2.muscache.com/im/pictures/5936fef0-b...,https://a2.muscache.com/im/pictures/5936fef0-b...,Roslindale,1,1,"['email', 'phone', 'facebook', 'reviews']",t,f,"Birch Street, Boston, MA 02131, United States",Roslindale,Roslindale
1,https://www.airbnb.com/users/show/2572247,Andrea,2012-06-07,"Boston, Massachusetts, United States",I live in Boston and I like to travel and have...,within an hour,100%,100%,f,https://a2.muscache.com/im/users/2572247/profi...,https://a2.muscache.com/im/users/2572247/profi...,Roslindale,1,1,"['email', 'phone', 'facebook', 'linkedin', 'am...",t,t,"Pinehurst Street, Boston, MA 02131, United States",Roslindale,Roslindale
2,https://www.airbnb.com/users/show/16701,Phil,2009-05-11,"Boston, Massachusetts, United States","I am a middle-aged, single male with a wide ra...",within a few hours,100%,88%,t,https://a2.muscache.com/im/users/16701/profile...,https://a2.muscache.com/im/users/16701/profile...,Roslindale,1,1,"['email', 'phone', 'reviews', 'jumio']",t,t,"Ardale St., Boston, MA 02131, United States",Roslindale,Roslindale
3,https://www.airbnb.com/users/show/6031442,Meghna,2013-04-21,"Boston, Massachusetts, United States",My husband and I live on the property. He’s a...,within a few hours,100%,50%,f,https://a2.muscache.com/im/pictures/5d430cde-7...,https://a2.muscache.com/im/pictures/5d430cde-7...,NaN,1,1,"['email', 'phone', 'reviews']",t,f,"Boston, MA, United States",NaN,Roslindale
4,https://www.airbnb.com/users/show/15396970,Linda,2014-05-11,"Boston, Massachusetts, United States",I work full time for a public school district....,within an hour,100%,100%,t,https://a0.muscache.com/im/users/15396970/prof...,https://a0.muscache.com/im/users/15396970/prof...,Roslindale,1,1,"['email', 'phone', 'reviews', 'kba']",t,t,"Durnell Avenue, Boston, MA 02131, United States",Roslindale,Roslindale


In [24]:
listings['last_scraped'].unique()

array(['2016-09-07'], dtype=object)

In [ ]:
listings.head()

In [ ]:
missing_values = listings.isna().sum()/listings.shape[0]*100
missing_values.sort_values(ascending = False)

In [ ]:
listings.shape

In [ ]:
#drop duplicates
listings.drop_duplicates(inplace=True)

In [ ]:
listings.shape

### Drop columns

In [ ]:
#There are 5 columns with almost all missing values -> we are dropping them
listings.drop(columns={'has_availability','neighbourhood_group_cleansed','license','jurisdiction_names','square_feet'},inplace=True,axis = 1)

In [ ]:
# drop the columns with the url - no need at this point
listings.drop(columns={'listing_url','thumbnail_url','medium_url','picture_url','xl_picture_url','availability_30','availability_60','availability_90','availability_365','maximum_nights','weekly_price','monthly_price','calendar_updated','host_picture_url','host_listings_count','neighbourhood'},inplace=True,axis = 1)

In [ ]:
listings.shape

In [ ]:
#find columns with the same value
col_list = []
for col in listings.columns:
    if listings[col].nunique()==1:
        col_list.append(col)

In [ ]:
col_list

In [ ]:
# drop the columns with the same value
listings.drop(columns={'scrape_id',
 'last_scraped',
 'experiences_offered',
 'state',
 'country_code',
 'country',
 'calendar_last_scraped',
 'requires_license'},inplace=True,axis = 1)

In [ ]:
#find columns that are dummy variables
col_list2 = []
for col in listings.columns:
    if listings[col].nunique()==2:
        col_list2.append(col)

In [ ]:
#find columns with the three values
col_list2

In [ ]:
listings['host_is_superhost'].value_counts()

In [ ]:
listings['host_has_profile_pic'].value_counts()

In [ ]:
listings['host_identity_verified'].value_counts()

In [ ]:
listings['is_location_exact'].value_counts()

In [ ]:
listings['instant_bookable'].value_counts()

In [ ]:
listings['require_guest_profile_picture'].value_counts()

In [ ]:
listings['require_guest_phone_verification'].value_counts()

In [ ]:
#Transform categorical variables into Binary
for col in col_list2:
    listings[col] = np.where(listings[col]=='t', 1, 0)

In [ ]:
#drop also column host_has_profile_pic, since the majority of the values are true
listings.drop('host_has_profile_pic',inplace=True, axis=1)

In [ ]:
listings.shape

In [ ]:
#for security deposit - my assumption is that when there is no value -> there is NO security deposit. Also for the cleaning fee - if omitted -> then there isn't one. THIS IS OK to be done before the SPLIT
listings['security_deposit'].fillna(0,inplace=True)
listings['cleaning_fee'].fillna(0,inplace=True)

In [ ]:
#The text data contained in space, summary, description is almost the same, so I will leave only the description, since it has less missing values
listings.drop(['space','summary','name','host_url','host_name','host_thumbnail_url'],axis=1,inplace=True)

In [ ]:
#column market contains very few values, and the majority of the values are only 1 value -> we will drop it
listings['market'].value_counts()

In [ ]:
listings.drop(['market'],axis=1,inplace=True)

In [ ]:
#For the rest of the text variables, since there is a big part of them missing, I will encode them as 1: with data, 0 - no data
col_list = ['notes', 'transit','access','interaction','house_rules','neighborhood_overview','host_about']
for col in col_list:
    listings[col].fillna(0,inplace=True)
    listings[col] = np.where(listings[col].notna()==True, 1, 0)

In [ ]:
listings.drop(columns={'notes', 'transit','access','interaction','house_rules','neighborhood_overview','host_about'},axis=1,inplace=True)

#Date variables we will transform into numerical:
- host_since: days between last_day_reserved and host_since - we will get days into operation of the listing
- last_review: days between last_review and last reservation
- first_review: days between last_reservation and first_review  - we will omit this, since it is very close to the first metrics

Binning and discretization should be done over the host response rate and host acceptance rate

In [ ]:
listings['host_since']= pd.to_datetime(listings['host_since'])
listings['first_review']= pd.to_datetime(listings['first_review'])
listings['last_review']= pd.to_datetime(listings['last_review'])

In [ ]:
cal_last_reservation = calendar[calendar['available']=='f'].groupby(['listing_id']).agg({'date':'max'}).reset_index()

In [ ]:
listings = listings.merge(cal_last_reservation, how = 'left', left_on = 'id',right_on = 'listing_id')

In [ ]:
date = pd.to_datetime('2016-09-07')
listings['days_operation'] = (date- listings['host_since']).dt.days
#listings['days_operation'].dtype()

In [ ]:
#new variable that gives the days between the last reservation and the last review
listings['days_review'] = (listings['date']- listings['last_review']).dt.days

In [ ]:
listings.drop(columns={'host_since','last_review','first_review'},inplace=True,axis=1)

In [ ]:
listings.drop(['date'],axis=1,inplace=True)

In [ ]:
listings.drop(['listing_id','zipcode'],axis=1,inplace=True)

In [ ]:
listings.describe(include='object').transpose()

In [ ]:
#Transform the price columns to integer values
cols = ['security_deposit','price','cleaning_fee','extra_people']
for col in cols:
    listings[col] = listings[col].str.replace('$','')
    listings[col] = listings[col].str.replace(',','')
    #listings[col_col] = listings[col].str.findall('\d')
    listings[col].fillna(0,inplace=True)
    listings[col] = pd.to_numeric(listings[col])

In [ ]:
listings['security_deposit'].head()

In [ ]:
listings['price'].head()

In [ ]:
col_objects = listings.select_dtypes('object').columns

In [ ]:
for col in col_objects:
    print (col, listings[col].isna().sum())

In [ ]:
#Map host location to 1:within Boston, 0 - not in Boston
listings['host_location'] = np.where(listings['host_location']=='Boston, Massachusetts, United States', 1, 0)
listings['host_location'].fillna(1,inplace=True)

In [ ]:
listings['host_response_time'].value_counts()

In [ ]:
listings.describe().transpose()

In [ ]:
listings[['price','accommodates','guests_included']].value_counts()

For the 5 categorical variables with missing values and for all the numerical categoricals with missing value we will populate with mean/median and the mode/for the categorical ones/
Before that, we need to study the distributions of the variables and perform EDA. 
Also check for the prerequisites of the Linear model /the assumptions/.
+ Feature selection techniques -> correlation, chi-square test<br>
Categorical Encoding<br>
__BEFORE ANY OTHER COMPUTATIONS, we need to split the dataset into TRAIN AND TEST set.__
We can perform GridSearch without Cross Validation.

In [ ]:
#as a result from the analysis in 02.1 notebook - dropping the description field
listings.drop(['description'],axis=1,inplace=True)

### Exploratory Data Analysis - distributions

In [ ]:
#sns.pairplot(listings)

In [ ]:
'''
Input: dataframe and variable to be analysed
Output: variable descriptive statistics
'''

def diagnostic_plots(df, variable):
    # function takes a dataframe (df) and
    # the variable of interest as arguments

    # define figure size
    plt.figure(figsize=(16, 4))

    # histogram
    plt.subplot(1, 3, 1)
    sns.histplot(df[variable], bins=30)
    plt.title('Histogram')

    # Q-Q plot
    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.ylabel('Variable quantiles')

    # boxplot
    plt.subplot(1, 3, 3)
    sns.boxplot(y=df[variable])
    plt.title('Boxplot')

    plt.show()

In [ ]:
diagnostic_plots(listings,'price')

Transform the price into price per person, as well as take into account the additional price for guests

In [ ]:
listings['price_per_person'] = round(listings['price']/(listings['accommodates']+listings['guests_included']),2)+listings['extra_people']

In [ ]:
listings.drop(['price','accommodates','guests_included','extra_people'],axis=1,inplace=True)

In [ ]:
diagnostic_plots(listings,'price_per_person')

In [ ]:
def find_skewed_boundaries(df, variable, distance):

    # Let's calculate the boundaries outside which sit the outliers
    # for skewed distributions

    # distance passed as an argument, gives us the option to
    # estimate 1.5 times or 3 times the IQR to calculate
    # the boundaries.

    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)

    lower_boundary = df[variable].quantile(0.25) - (IQR * distance)
    upper_boundary = df[variable].quantile(0.75) + (IQR * distance)

    return upper_boundary, lower_boundary

In [ ]:
# find limits for RM

RM_upper_limit, RM_lower_limit = find_skewed_boundaries(X_train, 'price_per_person', 1.5)
RM_upper_limit, RM_lower_limit

In [ ]:
IQR = listings['price_per_person'].quantile(0.75) - listings['price_per_person'].quantile(0.25)

In [ ]:
IQR

In [ ]:
# let's flag the outliers in the data set
listings['outliers_price'] = np.where(listings['price_per_person'] > RM_upper_limit, True,
                       np.where(listings['price_per_person']  < RM_lower_limit, True, False))

In [ ]:
listings['outliers_price'].value_counts()

In [ ]:
145/3585

In [ ]:
listings = listings[listings['outliers_price']==False].copy()

In [ ]:
listings_out.shape

In [ ]:
#drop the days_review column
listings.drop(columns=['days_review'],axis=1,inplace=True)

# 2. Missing values computations 
For the rest of the columns we will handle the missing values one by one
IT IS VERY IMPORTANT BEFORE IMPUTING MISSING VALUES TO SPLIT THE DATA INTO TRAIN&TEST -> TO AVOID DATA LEAKAGE. __COMPUTATIONS SHOULD BE DONE ONLY ON THE TRAIN SET!__<br>
Use sklearn SimpleImputer or other in-built imputers -> and place it in a Pipeline. 

In [ ]:
df = listings.drop(['price_per_person'],axis=1)
y = listings['price_per_person'].copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, # just the features
                                                    y, # the target
                                                    test_size=0.3, # the percentage of obs in the test set
                                                    random_state=42) # for reproducibility

In [ ]:
X_train['beds'].value_counts()

In [ ]:
X_test['beds'].value_counts()

In [ ]:
X_train['beds'].hist()

In [ ]:
X_train.hist(bins=50, figsize=(10,10))
plt.show()

In [ ]:
X_test.hist(bins=50, figsize=(10,10))
plt.show()

In [ ]:
y_train.hist()

In [ ]:
y_test.hist()

In [ ]:
listings.isna().sum().sort_values(ascending = False)

In [ ]:
X_train.describe().transpose()

In [ ]:
#We will use missing value for the review columns NAs, since the proportion of the missing data is huge - 20% of the total dataset
missing_value = ['reviews_per_month','review_scores_location',
                          'review_scores_value','review_scores_checkin','review_scores_cleanliness',
                          'review_scores_communication','review_scores_rating','review_scores_accuracy',
                'host_response_time']

for col in missing_value:
    X_train[col].fillna(-1,inplace=True)
    X_test[col].fillna(-1,inplace=True)

In [ ]:
cols = ['host_response_rate','host_acceptance_rate']
for col in cols:
    X_train[col] = X_train[col].str.replace('%','')
    X_train[col] = pd.to_numeric(X_train[col])
    X_test[col] = X_test[col].str.replace('%','')
    X_test[col] = pd.to_numeric(X_test[col])

In [ ]:
# Imputing with median value - Based on the nature of the variables, as well as on their distribution, we are choosing what to impute with: variables with more close to normal distribution - mean; skewed variables - median
numeric_features_median = ['bathrooms','bedrooms','beds','days_operation','host_response_rate','host_acceptance_rate']

for col in numeric_features_median:
    X_train[col].fillna(X_train[col].median(),inplace=True)
    X_test[col].fillna(X_test[col].median(),inplace=True)

In [ ]:
#impute with mode for the categorical features
cat_features_mode = ['host_neighbourhood','property_type','city']

for col in cat_features_mode:
    most = X_train[col].mode().values[0]
    X_train[col].fillna(most,inplace=True)
    most2 = X_test[col].mode().values[0]
    X_test[col].fillna(most2,inplace=True)

In [ ]:
X_train.isna().sum().sort_values(ascending=False).head()

In [ ]:
X_test.isna().sum().head()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
#In addition, we will drop the id, the host_id
X_train.drop(['id','host_id'],axis=1,inplace=True)
X_test.drop(['id','host_id'],axis=1,inplace=True)

In [ ]:
#Drop street, since we have geo coordinates
X_train.drop(['street'],axis=1,inplace=True)
X_test.drop(['street'],axis=1,inplace=True)

In [ ]:
X_train.describe().transpose()

In [ ]:
X_train.describe(include='object').transpose()

In [ ]:
X_train.shape

In [ ]:
X_train.drop(['host_location',
             'is_location_exact','calculated_host_listings_count'],axis=1,inplace=True)
X_test.drop(['host_location',
              'is_location_exact','calculated_host_listings_count'],axis=1,inplace=True)

# 3. Other feature engineering 
- cardinality check
- categorical encoding
- discretization

### 3.1. Cardinality check 
on columns: host_response_time, host_neighbourhood, host_verifications, neighbourhood_cleansed,city,smart_location,property type, room type, bed type, amenities, cancellation policy<br>

host_response_rate, neighbourhood_cleansed, property_type, room_type, bed_type, cancellation_policy - to be encoded<br>
city - encoded as Binary; smart_location - stores the same info as city, dropped<br>
host_neighbourhood - transform into another variable: 1 if it is the same with the listing neighbourhood, 0 if it is different<br>
__amenities & host_verifications - text preprocessing&feature vectorization

In [ ]:
X_train['host_response_time'].value_counts()

In [ ]:
X_train['smart_location'].value_counts()

In [ ]:
#THe prevailing value was Boston, so we marked the city with 1 for Boston, and 0 for the rest
X_train['city']=np.where(X_train['city']=='Boston',1,0)
X_test['city']=np.where(X_test['city']=='Boston',1,0)

In [ ]:
#drop smart location, as it is the same as City
X_train.drop(['smart_location'],axis=1,inplace=True)
X_test.drop(['smart_location'],axis=1,inplace=True)

In [ ]:
#For the host neighbourhoud, I will introduce a new variable - whether it is the same as the property neighbourhood. 
X_train['same_neigh'] = np.where(X_train['host_neighbourhood'].str.lower() == X_train['neighbourhood_cleansed'].str.lower(), 1, 0)

In [ ]:
X_test['same_neigh'] = np.where(X_test['host_neighbourhood'].str.lower() == X_test['neighbourhood_cleansed'].str.lower(), 1, 0)

In [ ]:
X_train.drop(['host_neighbourhood'],axis=1,inplace=True)
X_test.drop(['host_neighbourhood'],axis=1,inplace=True)

In [ ]:
X_train['neighbourhood_cleansed'].value_counts()

In [ ]:
X_train['amenities'].nunique()

In [ ]:
#X_train['host_verifications'].nunique()

In [ ]:
listings['property_type'].value_counts()

#### 3.1.1.Categorical handling of host verifications and amenities

In [ ]:
def text_cleaner_open_amenities(serie, words= []):
    '''
    input: pandas series, stop words
    ouput: removes special char, stop words, symbols.'''
    serie = serie.astype(str).str.lower()
    #st = PorterStemmer()
    stop= stopwords.words('english') + words
    stop = [x for x in stop]
    
    serie=serie.str.replace('"','')
    serie=serie.str.replace('{','')
    serie=serie.str.replace('}','')
    serie= serie.apply(lambda x: " ".join([word for word in x.split() 
                                           if word not in stop])) 
    return serie

In [ ]:
X_train['amenities'] = text_cleaner_open_amenities(X_train['amenities'])
X_test['amenities'] = text_cleaner_open_amenities(X_test['amenities'])

In [ ]:
vectorizer_am = CountVectorizer(tokenizer=lambda x: x.split(','))
vectorizer_am.fit(X_train['amenities'])

In [ ]:
vectorizer_am.get_feature_names()

In [ ]:
dt = vectorizer_am.transform(X_train['amenities'])

In [ ]:
dt.shape

In [ ]:
#new = pd.DataFrame(dt.toarray(), columns=vectorizer_am.get_feature_names(), index= a.index)

In [ ]:
#new.isna().sum()

In [ ]:
X_train = X_train.join(pd.DataFrame(dt.toarray(), columns=vectorizer_am.get_feature_names(), index= X_train.index))

In [ ]:
X_train.shape

In [ ]:
X_train.head()

In [ ]:
dt1 = vectorizer_am.transform(X_test['amenities'])

In [ ]:
dt1.shape

In [ ]:
X_test = X_test.join(pd.DataFrame(dt1.toarray(), columns=vectorizer_am.get_feature_names(), index= X_test.index))

In [ ]:
#combine some of the features
X_train.loc[X_train['wireless internet']==1,'internet']=1
X_test.loc[X_test['wireless internet']==1,'internet']=1

In [ ]:
X_train.loc[X_train['tv']==1,'cable tv']=1
X_test.loc[X_test['tv']==1,'cable tv']=1

In [ ]:
X_train.loc[X_train['washer / dryer']==1,'washer']=1
X_train.loc[X_train['washer / dryer']==1,'dryer']=1

X_test.loc[X_test['washer / dryer']==1,'washer']=1
X_test.loc[X_test['washer / dryer']==1,'dryer']=1

In [ ]:
X_train.drop(['','translation missing: en.hosting_amenity_49',
 'translation missing: en.hosting_amenity_50','wireless internet','tv','washer / dryer'],axis=1,inplace=True)

In [ ]:
X_test.drop(['','translation missing: en.hosting_amenity_49',
 'translation missing: en.hosting_amenity_50','wireless internet','tv','washer / dryer'],axis=1,inplace=True)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

#### 3.1.2.Host verifications

In [ ]:
def text_cleaner_open_verifications(serie, words= []):
    '''
    input: pandas series, stop words
    ouput: removes special char, stop words, symbols.'''
    serie = serie.astype(str).str.lower()
    #st = PorterStemmer()
    stop= stopwords.words('english') + words
    stop = [x for x in stop]
    
    serie=serie.str.replace('\'','')
    serie=serie.str.replace('{','')
    serie=serie.str.replace('}','')
    serie=serie.str.replace('[','')
    serie=serie.str.replace(']','')
    serie = serie.str.strip()
    serie= serie.apply(lambda x: " ".join([word for word in x.split() 
                                           if word not in stop])) 
    return serie

In [ ]:
X_train['host_verifications'] = text_cleaner_open_verifications(X_train['host_verifications'])
X_test['host_verifications'] = text_cleaner_open_verifications(X_test['host_verifications'])

In [ ]:
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(','))
vectorizer.fit(X_train['host_verifications'])

In [ ]:
vectorizer.get_feature_names()

In [ ]:
dt2 = vectorizer.transform(X_train['host_verifications'])

In [ ]:
dt2.shape

In [ ]:
X_train = X_train.join(pd.DataFrame(dt2.toarray(), columns=vectorizer.get_feature_names(), index= X_train.index))

In [ ]:
dt3 = vectorizer.transform(X_test['host_verifications'])

In [ ]:
dt3.shape

In [ ]:
X_test = X_test.join(pd.DataFrame(dt3.toarray(), columns=vectorizer.get_feature_names(), index= X_test.index))

In [ ]:
X_test.shape

In [ ]:
X_train.loc[X_train[' phone']==1,'phone']=1
X_test.loc[X_test[' phone']==1,'phone']=1

In [ ]:
X_train.drop(['phone'],axis=1,inplace=True)
X_test.drop(['phone'],axis=1,inplace=True)

In [ ]:
X_train.drop(['amenities','host_verifications'],axis=1,inplace=True)
X_test.drop(['amenities','host_verifications'],axis=1,inplace=True)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train_t=X_train.copy()
X_test_t=X_test.copy()

#### Discretization

In [ ]:
cols = ['host_response_rate','host_acceptance_rate']
for col in cols:
    X_train[col] = X_train[col].str.replace('%','')
    X_train[col] = pd.to_numeric(X_train[col])
    X_test[col] = X_test[col].str.replace('%','')
    X_test[col] = pd.to_numeric(X_test[col])

In [ ]:
X_train['host_response_rate'].fillna(-1,inplace=True)
X_train['host_acceptance_rate'].fillna(-1,inplace=True)

X_test['host_response_rate'].fillna(-1,inplace=True)
X_test['host_acceptance_rate'].fillna(-1,inplace=True)

In [ ]:
# bucket boundaries
buckets = [-1, 0, 20, 40, 50, 70, 90,100]
X_train['response_rate_buckets'] = pd.cut(X_train['host_response_rate'], bins=buckets, include_lowest=True)
X_test['response_rate_buckets'] = pd.cut(X_test['host_response_rate'], bins=buckets, include_lowest=True)

In [ ]:
# bucket 2 boundaries
buckets2 = [-1, 0, 20, 40, 50, 70, 90,100]
X_train['accept_rate_buckets'] = pd.cut(X_train['host_acceptance_rate'], bins=buckets, include_lowest=True)
X_test['accept_rate_buckets'] = pd.cut(X_test['host_acceptance_rate'], bins=buckets, include_lowest=True)

In [ ]:
X_train['response_rate_buckets'].value_counts()

In [ ]:
X_test['response_rate_buckets'].value_counts()

In [ ]:
X_train.drop(['host_response_rate','host_acceptance_rate'],axis=1,inplace=True)

In [ ]:
X_test.drop(['host_response_rate','host_acceptance_rate'],axis=1,inplace=True)

### 3.2. Categorical encoding
For this I will use one-hot encoding. 
__For Linear regression model - encode into K-1 variables<br>
For tree models - encode into K variables, since with the ensemble methods we are not using all features__
I am using the One-hot encoder from the feature engine, since it directly returns a dataframe w/o the original variable, with the respective names etc.

In [ ]:
X_train.describe(include='object').transpose()

In [ ]:
X_train.isna().sum().head()

In [ ]:
ohe_enc = fe_OneHotEncoder(
    top_categories=None,
    variables=['host_response_time','neighbourhood_cleansed','property_type','room_type','bed_type','cancellation_policy','response_rate_buckets','accept_rate_buckets'],  # we can select which variables to encode
    drop_last=False)  # to return k-1, false to return k

ohe_enc.fit(X_train)

In [ ]:
X_train_tr = ohe_enc.transform(X_train)

X_train_tr.head()

In [ ]:
X_train_tr.shape

In [ ]:
X_test_tr = ohe_enc.transform(X_test)

X_test_tr.head()

In [ ]:
X_test_tr.shape

In [ ]:
X_test_tr.isna().sum().sort_values(ascending=False).head()

All variables are transformed into numerical, encoded, and without null values!

In [ ]:
X_train_tr.info()

In [ ]:
X_test_tr.info()

# 4. Feature selection

## 4.1. Constant features and quasi-constant features

In [ ]:
constant_features = [
    feat for feat in X_train_tr.columns if X_train_tr[feat].std() == 0
]

In [ ]:
constant_features

In [ ]:
#drop the constant features
X_train_tr.drop(constant_features,axis=1,inplace=True)
X_test_tr.drop(constant_features,axis=1,inplace=True)

In [ ]:
# remove quasi-constant features
sel = VarianceThreshold(
    threshold=0.01)  # 0.1 indicates 99% of observations approximately

sel.fit(X_train_tr)  # fit finds the features with low variance

sum(sel.get_support()) # how many not quasi-constant?

In [ ]:
X_train_tr.shape

In [ ]:
features_not_keep = X_train_tr.columns[~sel.get_support()]

In [ ]:
features_not_keep 

I will not remove the quasi-constant features for now

## 4.2. Remove correlated features

In [ ]:
# find and remove correlated features

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    
    corr_matrix = dataset.corr()
    
    for i in range(len(corr_matrix.columns)):
    
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X_train_tr, 0.8)
print('correlated features: ', len(set(corr_features)) )

In [ ]:
corr_features

In [ ]:
X_train_tr.shape

In [ ]:
X_test_tr.shape

## Remove outliers from price
Only from tH TRAIN SET!

In [ ]:
train = X_train_tr.join(y_train)

In [ ]:
train.head()

In [ ]:
RM_upper_limit, RM_lower_limit = find_skewed_boundaries(train, 'price_per_person', 1.5)
RM_upper_limit, RM_lower_limit

In [ ]:
train['outliers_price'] = np.where(train['price_per_person'] > RM_upper_limit, True,
                       np.where(train['price_per_person'] < RM_lower_limit, True, False))

In [ ]:
train['outliers_price'].value_counts()

In [ ]:
101/2509

In [ ]:
train = train[train['outliers_price']==False]

In [ ]:
train.shape

In [ ]:
X_train_tr = train.drop('price_per_person',axis=1)
y_train = train['price_per_person'].copy()

In [ ]:
X_train_tr.drop(columns={'outliers_price'},axis=1,inplace=True)

In [ ]:
X_train_tr.shape

In [ ]:
X_test_tr.shape

In [ ]:
y_train.shape

# 5. Feature scaling
Before the modelling part, and depending on the model itself. E.g. tree models do not require feature scaling

In [ ]:
y_test.head()

In [ ]:
rf = RandomForestRegressor(n_estimators=300, random_state=42)
rf.fit(X_train_tr, y_train)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train_tr)

X_train_scaled = scaler.transform(X_train_tr)
X_test_scaled = scaler.transform(X_test_tr)

In [ ]:
pred = rf.predict(X_train_tr)

In [ ]:
pred_test = rf.predict(X_test_tr)

In [ ]:
#R 2 based on training set
r2_score(y_train,pred)

In [ ]:
from math import sqrt
sqrt(mean_squared_error(y_train,pred))

In [ ]:
#R 2 based on training set
r2_score(y_test,pred_test)

In [ ]:
sqrt(mean_squared_error(y_test,pred_test))

In [ ]:
rf.feature_importances_

In [ ]:
feat_importances = pd.Series(rf.feature_importances_, index=X_train_tr.columns)
feat_importances.nlargest(15).plot(kind='barh')
plt.title("Top 15 important features")
plt.show()

In [ ]:
rf1 = RandomForestRegressor(n_estimators=300, random_state=42)
rf1.fit(X_test_tr, y_test)

In [ ]:
pred = rf1.predict(X_test_tr)

In [ ]:
#R 2 based on training set
r2_score(y_test,pred)

In [ ]:
feat_importances = pd.Series(rf1.feature_importances_, index=X_test_tr.columns)
feat_importances.nlargest(15).plot(kind='barh')
plt.title("Top 15 important features")
plt.show()

In [ ]:
X_test_tr['days_operation'].min()

In [ ]:
X_train_tr['days_operation'].min()

In [ ]:
X_train['property_type'].value_counts()/X_train.shape[0]

In [ ]:
X_test['property_type'].value_counts()/X_test.shape[0]

In [ ]:
X_train_tr.info()

In [ ]:
diagnostic_plots(y_train,)

In [ ]:
lr = LinearRegression()
lr.fit(X_train_scaled,y_train)

In [ ]:
pred = lr.predict(X_train_scaled)
pred_test_lr = lr.predict(X_test_scaled)

In [ ]:
#R 2 based on training set
r2_score(y_train,pred)

In [ ]:
from math import sqrt
sqrt(mean_squared_error(y_train,pred))

In [ ]:
#R 2 based on training set
r2_score(y_test,pred_test_lr)

In [ ]:
sqrt(mean_squared_error(y_test,pred_test_lr))

In [ ]:
#Visualize model prediction
def model_prediction_plot(y_test, pred, name_test, name_pred, Title):
    
    plt.figure(figsize=(10, 8))
    ax1 = sns.distplot(y_test, hist=False, color="r", label=name_test)
    ax2 = sns.distplot(pred, hist=False, color="b", label=name_pred, ax=ax1)
    plt.title(Title)
    plt.xlabel('Price (dollars)')
    plt.legend()
    plt.show()
    plt.close()
model_prediction_plot(y_test, pred_test, 'Actual Values', 'Predicted Values', 
                 'Distribution  Plot of  Predicted Value vs Test Data Distribution')